<a href="https://colab.research.google.com/github/Tong00020/PLN---2022/blob/main/Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Atividade 3: Recuperação de Textos

Feito por:
Lucas De Almeida Bandeira Macedo 190047089
Tong Zhou 190038764

In [3]:
import nltk
import pandas as pd
import re
import numpy as np
import math

nltk.download('reuters')
from nltk.corpus import reuters

cats = reuters.categories()
print("Reuters has %d categories:\n%s" % (len(cats), cats))

fileids = reuters.fileids()

ModuleNotFoundError: No module named 'nltk'

In [2]:
categories = []
text = []

for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})
df

,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


In [4]:
texts = df['text']
texts

NameError: name 'df' is not defined

In [4]:
def preprocessing(texts):
  t = re.sub(r'#','',texts)
  t = re.sub(r':\)&','happy',t)
  t = t.lower()
  t = re.sub(r'[\,\+\=\-\_\?\[\]\{\}\@\.\'\$\!\&\>\<\;\"\(\)\:\/\\]+','',t)
  t = re.sub(r'[0-9]+','',t)
  return t

In [5]:
corpus = []

for text in texts:
  ptext = preprocessing(text)
  corpus.append(ptext)
# corpus

In [6]:
bow_ponderado = {}
N = {} # quantidade de termos em cada documento

for i,doc in enumerate(corpus):
    bow_ponderado['texto {}'.format(i+1)] = dict()
    N['texto {}'.format(i+1)] = len(doc.split())
    for word in doc.lower().split():
        if word in bow_ponderado['texto {}'.format(i+1)]:
            bow_ponderado['texto {}'.format(i+1)][word] += 1
        else:
            bow_ponderado['texto {}'.format(i+1)][word] = 1

for doc in bow_ponderado:
    for word in bow_ponderado[doc]:
        bow_ponderado[doc][word] /= N[doc]

bow_ponderado

{'texto 1': {'asian': 0.002844950213371266,
  'exporters': 0.004267425320056899,
  'fear': 0.001422475106685633,
  'damage': 0.002844950213371266,
  'from': 0.004267425320056899,
  'usjapan': 0.001422475106685633,
  'rift': 0.001422475106685633,
  'mounting': 0.001422475106685633,
  'trade': 0.021337126600284494,
  'friction': 0.001422475106685633,
  'between': 0.005689900426742532,
  'the': 0.05263157894736842,
  'us': 0.025604551920341393,
  'and': 0.02275960170697013,
  'japan': 0.012802275960170697,
  'has': 0.004267425320056899,
  'raised': 0.001422475106685633,
  'fears': 0.001422475106685633,
  'among': 0.002844950213371266,
  'many': 0.001422475106685633,
  'of': 0.04267425320056899,
  'asias': 0.001422475106685633,
  'exporting': 0.001422475106685633,
  'nations': 0.001422475106685633,
  'that': 0.007112375533428165,
  'row': 0.001422475106685633,
  'could': 0.001422475106685633,
  'inflict': 0.001422475106685633,
  'farreaching': 0.001422475106685633,
  'economic': 0.00284495

In [7]:
frase = " ".join(corpus)
tokens = frase.split()
vocab = sorted(set(tokens))
pd.Series(vocab).to_csv("vocabulario.csv")

In [8]:
idf = {}
for i, sent in enumerate(corpus):
  for wrd in set(sent.lower().split()):
    if wrd not in idf:
      idf[wrd] = 1
    else:
      idf[wrd] += 1
# for word in idf:
idf = pd.Series(idf)
idf = np.log(len(corpus)/idf)
idf

purpose       5.730842
this          1.664015
matter        5.066682
diplomatic    5.884992
many          3.609436
                ...   
genentech     9.286190
ltgenencor    9.286190
genencor      9.286190
genecor       9.286190
ltkrn         9.286190
Length: 34837, dtype: float64

In [9]:
df_term_frequency = pd.DataFrame().from_records(bow_ponderado).fillna(0).T
df_term_frequency

,asian,exporters,fear,damage,from,usjapan,rift,mounting,trade,friction,...,ltvaln,draugen,northermost,singleleg,gravitybase,subsea,buoyloading,ddrinks,conjecture,practise
texto 1,0.002845,0.004267,0.001422,0.002845,0.004267,0.001422,0.001422,0.001422,0.021337,0.001422,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
texto 10,0.000000,0.000000,0.000000,0.000000,0.006024,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
texto 100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
texto 1000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
texto 10000,0.000000,0.000000,0.000000,0.000000,0.008850,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
texto 9995,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
texto 9996,0.000000,0.000000,0.000000,0.000000,0.005376,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.021505,0.005376,0.005376,0.005376,0.005376,0.005376,0.000000,0.000000,0.000000
texto 9997,0.000000,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006173,0.006173,0.000000
texto 9998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006944


In [10]:
df_term_frequency = df_term_frequency.multiply(idf)
df_term_frequency

,a,aa,aaa,aabex,aachener,aaminus,aancor,aap,aaplus,aaps,...,zuccherifici,zuckerman,zuheir,zulia,zurich,zurichbased,zuyuan,zverev,zwermann,zzzz
texto 1,0.011726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 10,0.010759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 100,0.023385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 1000,0.024156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 10000,0.009726,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
texto 9995,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 9996,0.011818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 9997,0.013569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
texto 9998,0.007632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
def cosine_similarity(vetor1,vetor2):
    
    prod_interno = (vetor1*vetor2).sum()

    norma_vetor1 = math.sqrt((vetor1*vetor1).sum())
    norma_vetor2 = math.sqrt((vetor2*vetor2).sum())

    return prod_interno/(norma_vetor1*norma_vetor2)

In [85]:
n_doc = df_term_frequency.shape[0]
# n_doc = 2
ranking = [0]*10
for j in range(n_doc):
    res = []
    for i in range(df_term_frequency.shape[0]):
        res.append(cosine_similarity(df_term_frequency.loc[f"texto {j+1}"], df_term_frequency.loc[f"texto {i+1}"]))

    top_10000 = pd.Series(res).sort_values(ascending=False)

    top_10 = top_10000.head(11)
    cat_orig = df.iloc[top_10.index[0]].categories[0]

    for i in range(1, 11):
        ranking[i-1] += cat_orig in df.iloc[top_10.index[i]].categories

ranking

[2, 2, 2, 1, 1, 2, 1, 2, 2, 1]

In [87]:
acc = 0
for number in ranking:
  acc += number/n_doc

acc /= 10
acc

0.8

In [80]:
pd.Series(ranking).to_csv("output.csv")